# Graficos Validaco MGB

> Notebook organizado para reprodutibilidade. Edite apenas a célula **CONFIGURAÇÕES**.

In [ ]:
from pathlib import Path
import os

# CONFIGURAÇÕES (edite se necessário)
# A pasta raiz do projeto (por padrão, a pasta acima de /notebooks)
ROOT = Path(os.getenv('CLIMBRA_PROJECT_ROOT', Path.cwd().parent)).resolve()
DATA_DIR = ROOT / 'data'
RAW_DIR  = DATA_DIR / '00_raw'
INT_DIR  = DATA_DIR / '01_intermediate'
FINAL_DIR= DATA_DIR / '02_final'
OUT_DIR  = ROOT / 'outputs'
FIG_DIR  = OUT_DIR / 'figures'
TAB_DIR  = OUT_DIR / 'tables'

for d in [RAW_DIR, INT_DIR, FINAL_DIR, FIG_DIR, TAB_DIR]:
    d.mkdir(parents=True, exist_ok=True)


In [ ]:
# ------------------------------------------------------------
# Script: Avaliação de Desempenho do MGB com Vazões Observadas
# ------------------------------------------------------------
# Objetivo:
# Ler séries temporais de vazões observadas e simuladas pelo modelo MGB,
# calcular métricas de desempenho e gerar dois gráficos:
# (1) Comparação temporal das séries;
# (2) Curva de permanência em escala logarítmica.
#
# Métricas calculadas:
# - NSE (Nash-Sutcliffe Efficiency)
# - NSE Logarítmico
# - RMSE (Erro Quadrático Médio)
# - BIAS (%)
# - Erro relativo no Q90 (%)
# - Erro relativo no Q95 (%)
#
# Entradas:
# - Arquivo de vazão observada (.txt, com colunas dia, mês, ano, valor)
# - Arquivo de vazão simulada do MGB (.txt, formato SIM_MC_XXXX.TXT)
#
# Saídas:
# - Arquivo PNG com gráfico de séries temporais
# - Arquivo PNG com gráfico da curva de permanência
#
# Autor: Matheus Marinho
# Data: Junho/2025
# ------------------------------------------------------------

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Definir período de análise "aaaa-mm-dd"
data_inicio = "1980-01-01"
data_fim = "1990-12-31"

# Definir nomes dos arquivos de entrada e saída
codigo_estacao = "65035000"
codigo_mini = "832"
grafico_saida = "Porto Amazonas_calib"
grafico_curva_saida = "Porto Amazonas_curva_perm"

# Caminhos completos
caminho_input_obs = r"C:\Users\Matheus Marinho\Desktop\IGUAÇU_OTTO\3_Estações FLU\Input\ascii_mgb/"
caminho_input_sim = r"C:\Users\Matheus Marinho\Desktop\IGUAÇU_OTTO\6_Calibração\2_Projeto\Output/"
caminho_output = r"C:\Users\Matheus Marinho\Desktop\IGUAÇU_OTTO\8_Resultados\Calibração_MGB/"

caminho_obs = f"{caminho_input_obs}{codigo_estacao}.txt"
caminho_sim = f"{caminho_input_sim}SIM_MC_{codigo_mini}.txt"
caminho_saida = f"{caminho_output}{grafico_saida}.png"
caminho_curva_saida = f"{caminho_output}{grafico_curva_saida}.png"

def ler_arquivo_txt(caminho, tipo="observado"):
    try:
        df = pd.read_csv(caminho, sep=r'\s+', engine='python', header=None)
        df.columns = ["dia", "mes", "ano", "valor"]
    except Exception as e:
        print(f"❌ Erro ao ler o arquivo {caminho}: {e}")
        return None

    print(f"\n📂 Lendo arquivo ({tipo}): {caminho}")
    print("🔍 Pré-visualização do conteúdo lido:")
    #print(df.head())

    # Verificação explícita
    if not all(col in df.columns for col in ["dia", "mes", "ano", "valor"]):
        print("❌ As colunas esperadas não foram encontradas.")
        return None

    # Conversões
    for col in ["dia", "mes", "ano"]:
        df[col] = pd.to_numeric(df[col], errors="coerce")
    df["valor"] = pd.to_numeric(df["valor"], errors="coerce")

    df.loc[df["valor"] == -1, "valor"] = np.nan

    if df[["ano", "mes", "dia"]].isna().any().any():
        print(f"\n❌ Erro: Algumas datas estão incompletas no arquivo ({tipo})!")
        #print(df[df[["ano", "mes", "dia"]].isna().any(axis=1)].head())
        return None

    # Agora cria coluna de data com nomes que o pandas espera
    df["data"] = pd.to_datetime(
    df.rename(columns={"ano": "year", "mes": "month", "dia": "day"})[["year", "month", "day"]],
    errors="coerce"
)
    df.set_index("data", inplace=True)

    return df


def calcular_metricas(obs, sim):
    df = pd.DataFrame({"obs": obs, "sim": sim}).dropna()

    if len(df) == 0:
        print("❌ Erro: Nenhuma observação válida após limpeza.")
        return None

    nse = 1 - (np.sum((df["obs"] - df["sim"]) ** 2) / np.sum((df["obs"] - df["obs"].mean()) ** 2))
    nse_log = 1 - (np.sum((np.log(df["obs"].replace(0, np.nan)) - np.log(df["sim"].replace(0, np.nan))) ** 2) /
                    np.sum((np.log(df["obs"].replace(0, np.nan)) - np.log(df["obs"].replace(0, np.nan)).mean()) ** 2))
    rmse = np.sqrt(np.mean((df["obs"] - df["sim"]) ** 2))
    bias = np.mean(df["obs"] - df["sim"]) / np.mean(df["obs"]) * 100

    q90_obs = np.percentile(df["obs"].dropna(), 10)
    q90_sim = np.percentile(df["sim"].dropna(), 10)
    erro_q90 = ((q90_sim - q90_obs) / q90_obs) * 100

    q95_obs = np.percentile(df["obs"].dropna(), 5)
    q95_sim = np.percentile(df["sim"].dropna(), 5)
    erro_q95 = ((q95_sim - q95_obs) / q95_obs) * 100

    return {"NSE": nse, "NSE Log": nse_log, "RMSE": rmse, "BIAS (%)": bias, "Δ Q90 (%)": erro_q90, "Δ Q95 (%)": erro_q95}

def plotar_series(df_obs, df_sim, metricas, caminho_saida=None):
    df_merged = df_obs.join(df_sim, lsuffix="_obs", rsuffix="_sim", how="inner")

    plt.figure(figsize=(12, 5))
    plt.plot(df_merged.index, df_merged["valor_obs"], label="Observado", color="black", linewidth=1.5)
    plt.plot(df_merged.index, df_merged["valor_sim"], label="Simulado", color="red", linestyle="dashed")

    plt.xlabel("")
    plt.ylabel("Vazão (m³/s)")
    plt.title("Verificação entre as Vazões observadas e simuladas")
    plt.legend(loc="lower center", bbox_to_anchor=(0.5, -0.2), ncol=2, frameon=False)
    plt.grid()

    texto_metricas = "\n".join([f"{k}: {v:.3f}" for k, v in metricas.items()])
    plt.text(0.02, 0.98, texto_metricas, transform=plt.gca().transAxes, fontsize=10,
             verticalalignment='top', bbox=dict(facecolor='white', alpha=0.8))

    plt.tight_layout()

    if caminho_saida:
        plt.savefig(caminho_saida, dpi=300, bbox_inches="tight")
        print(f"📁 Gráfico salvo em: {caminho_saida}")

    plt.show()

def plotar_curva_permanencia(df_obs, df_sim, caminho_saida=None):
    plt.figure(figsize=(10, 6))

    for df, label, color in zip([df_obs, df_sim], ["Observado", "Simulado"], ["black", "red"]):
        valores_ordenados = np.sort(df["valor"].dropna())[::-1]
        excedencia = np.linspace(0, 100, len(valores_ordenados))
        plt.plot(excedencia, valores_ordenados, label=label, color=color)

    q90_obs = np.percentile(df_obs["valor"].dropna(), 10)
    q90_sim = np.percentile(df_sim["valor"].dropna(), 10)
    plt.plot([], [], color="black", linestyle="dashed", label=f"Q90 Obs: {q90_obs:.2f}")
    plt.plot([], [], color="red", linestyle="dashed", label=f"Q90 Sim: {q90_sim:.2f}")

    plt.xlabel("Excedência (%)")
    plt.ylabel("Vazão (m³/s)")
    plt.yscale("log")
    plt.title("Curva de Permanência")
    plt.legend()
    plt.grid()

    if caminho_saida:
        plt.savefig(caminho_saida, dpi=300, bbox_inches="tight")
        print(f"📁 Curva de permanência salva em: {caminho_saida}")

    plt.show()

# Processamento principal
df_obs = ler_arquivo_txt(caminho_obs, tipo="observado")
df_sim = ler_arquivo_txt(caminho_sim, tipo="simulado")

if df_obs is not None and df_sim is not None:
    print("\n✅ Arquivos lidos com sucesso!")
    df_obs_filtrado = df_obs.loc[data_inicio:data_fim]
    df_sim_filtrado = df_sim.loc[data_inicio:data_fim]

    df_obs_filtrado["valor"] = df_obs_filtrado["valor"].replace(0, 1e-2)
    df_sim_filtrado["valor"] = df_sim_filtrado["valor"].replace(0, 1e-2)

    metricas = calcular_metricas(df_obs_filtrado["valor"], df_sim_filtrado["valor"])

    print("\n📊 Métricas de desempenho:")
    for chave, valor in metricas.items():
        print(f"   {chave}: {valor:.4f}")

    plotar_series(df_obs_filtrado, df_sim_filtrado, metricas, caminho_saida=caminho_saida)
    plotar_curva_permanencia(df_obs_filtrado, df_sim_filtrado, caminho_saida=caminho_curva_saida)

In [ ]:
# ------------------------------------------------------------
# Script: Geração Automática de Gráficos de Calibração do MGB
# ------------------------------------------------------------
# sub-bacias, calcular métricas de desempenho e gerar:
# (1) Gráfico de comparação temporal
# (2) Curva de permanência
#
# Entradas:
# - Um CSV de mapeamento com as colunas:
#   estacao_obs;codigo_mini;nome_estacao
# - Arquivos de vazão observada (.txt): ascii_mgb/<estacao_obs>.txt
# - Arquivos de vazão simulada do MGB: Output/SIM_MC_<codigo_mini>.TXT
#
# Saídas:
# - PNG com gráfico de séries temporais
# - PNG com curva de permanência
# - CSV com métricas por estação
#
# Autor: Matheus Marinho
# Data: Junho/2025
# ------------------------------------------------------------

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# === Parâmetros gerais ===
data_inicio     = "1980-01-01"
data_fim        = "1990-12-31"

# === Caminhos principais ===
caminho_input_obs = Path(r"E:\IGUAÇU_OTTO\3_Estações FLU\Input\ascii_mgb")
caminho_input_sim = Path(r"E:\IGUAÇU_OTTO\6_Calibração\2_Projeto\Output")
caminho_output    = Path(r"E:\IGUAÇU_OTTO\6_Calibração\Calibração_MGB")
mapeamento_csv    = Path(r"E:\IGUAÇU_OTTO\3_Estações FLU\Estações_mini.csv")

# === Função de leitura ===
def ler_arquivo_txt(caminho, tipo="observado"):
    try:
        df = pd.read_csv(caminho, sep=r'\s+', header=None, names=["dia", "mes", "ano", "valor"], dtype=str)

        # Remove vírgulas como separador de milhar
        df["valor"] = (
            df["valor"]
            .str.replace(",", "", regex=False)  # remove separador de milhar
            .astype(float)
        )

        # Substitui -1 por NaN
        df.loc[df["valor"] == -1, "valor"] = np.nan

    except Exception as e:
        print(f"❌ Erro ao ler {tipo} em {caminho}: {e}")
        return None

    # Converte data
    df["data"] = pd.to_datetime(
        df.rename(columns={"ano": "year", "mes": "month", "dia": "day"})[["year", "month", "day"]],
        errors="coerce"
    )
    df.set_index("data", inplace=True)

    if df.index.isna().any():
        print(f"❌ {tipo.capitalize()} possui datas inválidas.")
        return None

    return dfObjetivo:
# Automatizar a leitura das séries observadas e simuladas de múltiplas
# 

# === Função para calcular métricas ===
def calcular_metricas(obs, sim):
    df = pd.DataFrame({"obs": obs, "sim": sim}).dropna()
    if df.empty:
        return None

    nse = 1 - ((df["obs"] - df["sim"])**2).sum() / ((df["obs"] - df["obs"].mean())**2).sum()
    nse_log = 1 - ((np.log(df["obs"]) - np.log(df["sim"]))**2).sum() / ((np.log(df["obs"]) - np.log(df["obs"]).mean())**2).sum()
    #rmse = np.sqrt(((df["obs"] - df["sim"])**2).mean())
    bias = ((df["sim"] - df["obs"]).mean() / df["obs"].mean()) * 100
    q90_obs, q90_sim = np.percentile(df["obs"], 10), np.percentile(df["sim"], 10)
    q95_obs, q95_sim = np.percentile(df["obs"], 5),  np.percentile(df["sim"], 5)

    return {
        "NSE": nse,
        "NSE Log": nse_log,
        #"RMSE": rmse,
        "BIAS (%)": bias,
        "Δ Q90 (%)": (q90_sim - q90_obs) / q90_obs * 100,
        "Δ Q95 (%)": (q95_sim - q95_obs) / q95_obs * 100,
    }

# === Plot de séries temporais ===
def plotar_series(df_obs, df_sim, metricas, caminho_saida):
    df_merged = df_obs.join(df_sim, lsuffix="_obs", rsuffix="_sim", how="inner")

    plt.figure(figsize=(12, 5))
    plt.plot(df_merged.index, df_merged["valor_obs"], label="Observado", color="black")
    plt.plot(df_merged.index, df_merged["valor_sim"], label="Simulado", color="red", linestyle="--")
    plt.ylabel("Vazão (m³/s)")
    plt.title("Séries Temporais Observada vs Simulada")
    plt.grid()
    plt.legend()

    texto = "\n".join([f"{k}: {v:.2f}" for k, v in metricas.items()])
    plt.text(0.01, 0.95, texto, transform=plt.gca().transAxes,
             bbox=dict(facecolor="white", alpha=0.8), verticalalignment='top')

    plt.tight_layout()
    plt.savefig(caminho_saida, dpi=300)
    plt.close()

# === Plot da curva de permanência ===
def plotar_curva_permanencia(df_obs, df_sim, caminho_saida):
    plt.figure(figsize=(10, 5))
    for df, label, color in zip([df_obs, df_sim], ["Observado", "Simulado"], ["black", "red"]):
        dados = np.sort(df["valor"].dropna())[::-1]
        excedencia = np.linspace(0, 100, len(dados))
        plt.plot(excedencia, dados, label=label, color=color)

    plt.yscale("log")
    plt.xlabel("Excedência (%)")
    plt.ylabel("Vazão (m³/s)")
    plt.title("Curva de Permanência")
    plt.grid()
    plt.legend()
    plt.tight_layout()
    plt.savefig(caminho_saida, dpi=300)
    plt.close()

# === Execução principal ===
df_map = pd.read_csv(mapeamento_csv, sep=";", dtype=str)
todas_metricas = []

for _, row in df_map.iterrows():
    estacao_obs = row['estacao_obs']
    codigo_mini = row['codigo_mini']
    nome_estacao = row['nome_estacao'].replace(" ", "_")

    caminho_obs = caminho_input_obs / f"{estacao_obs}.txt"
    caminho_sim = caminho_input_sim / f"SIM_MC_{codigo_mini}.TXT"
    periodo_str = f"{data_inicio[:4]}_{data_fim[:4]}"
    saida_grafico = caminho_output / f"{nome_estacao}_calib_{periodo_str}.png"
    saida_curva   = caminho_output / f"{nome_estacao}_curva_perm_{periodo_str}.png"


    print(f"\n📂 Processando estação: {nome_estacao}")

    df_obs = ler_arquivo_txt(caminho_obs, tipo="observado")
    df_sim = ler_arquivo_txt(caminho_sim, tipo="simulado")

    if df_obs is None or df_sim is None:
        print(f"⚠️ Pulando {nome_estacao} por erro de leitura.")
        continue

    df_obs = df_obs.loc[data_inicio:data_fim]
    df_sim = df_sim.loc[data_inicio:data_fim]

    df_obs["valor"] = df_obs["valor"].replace(0, 1e-2)
    df_sim["valor"] = df_sim["valor"].replace(0, 1e-2)

    metricas = calcular_metricas(df_obs["valor"], df_sim["valor"])
    if metricas is None:
        print(f"⚠️ Sem dados válidos em {nome_estacao}")
        continue

    print("📊 Métricas:")
    for k, v in metricas.items():
        print(f"   {k}: {v:.2f}")

    # Armazena métricas
    linha = {"estacao_obs": estacao_obs, "codigo_mini": codigo_mini, "nome_estacao": nome_estacao}
    linha.update(metricas)
    todas_metricas.append(linha)

    plotar_series(df_obs, df_sim, metricas, saida_grafico)
    plotar_curva_permanencia(df_obs, df_sim, saida_curva)
    print(f"✅ Gráficos salvos para: {nome_estacao}")

# === Salvamento final das métricas ===
df_metricas = pd.DataFrame(todas_metricas)
periodo_str = f"{data_inicio[:4]}_{data_fim[:4]}"
arquivo_metricas = caminho_output / f"metricas_calibracao_{periodo_str}.csv"
df_metricas.to_csv(arquivo_metricas, sep=";", index=False)
print(f"\n📁 Arquivo de métricas salvo em: {arquivo_metricas}")


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from pathlib import Path

# ================== CONFIG ==================
periodos = {"Validação": ("1980-01-01", "1990-12-31")}

inp_obs = Path(r"E:\IGUAÇU_OTTO\3_Estações FLU\Input\ascii_mgb")
inp_sim = Path(r"E:\IGUAÇU_OTTO\6_Calibração\2_Projeto\Output")
outdir  = Path(r"E:\IGUAÇU_OTTO\6_Calibração\Calibração_MGB")
map_csv = Path(r"E:\IGUAÇU_OTTO\3_Estações FLU\Estações_mini.csv")
map_img = Path(r"E:\IGUAÇU_OTTO\Mapa 05 - Minibacias.png")

outdir.mkdir(parents=True, exist_ok=True)

# tamanho físico do painel
W_CM, H_CM = 25.0, 5.0
DPI = 400

# fontes
FS_TITLE, FS_LAB, FS_TICK, FS_STAT = 9.0, 7.0, 6.0, 6.0

cm2in = lambda cm: cm/2.54

def read_txt(path):
    df = pd.read_csv(path, sep=r"\s+", header=None, names=["d","m","y","q"], dtype=str)
    df["q"] = df["q"].str.replace(",", "", regex=False).astype(float)
    df.loc[df["q"] == -1, "q"] = np.nan
    dt = pd.to_datetime(df.rename(columns={"y":"year","m":"month","d":"day"})[["year","month","day"]], errors="coerce")
    return df.assign(date=dt).set_index("date")[["q"]].sort_index()

def metrics(obs, sim):
    df = pd.DataFrame({"obs": obs, "sim": sim}).dropna()
    df = df[(df.obs > 0) & (df.sim > 0)]
    if df.empty: return None
    nse = 1 - ((df.obs-df.sim)**2).sum() / ((df.obs-df.obs.mean())**2).sum()
    nseL = 1 - ((np.log(df.obs)-np.log(df.sim))**2).sum() / ((np.log(df.obs)-np.log(df.obs).mean())**2).sum()
    bias = (df.sim.mean()-df.obs.mean())/df.obs.mean()*100
    q90o,q90s = np.percentile(df.obs,10), np.percentile(df.sim,10)
    q95o,q95s = np.percentile(df.obs, 5), np.percentile(df.sim, 5)
    return {"NSE":float(nse), "NSE Log":float(nseL), "BIAS (%)":float(bias),
            "Δ Q90 (%)":float((q90s-q90o)/q90o*100), "Δ Q95 (%)":float((q95s-q95o)/q95o*100)}

def fdc(x):
    v = np.sort(x.dropna().values)[::-1]
    p = np.linspace(0, 100, len(v))
    return p, v

# ================== RUN ==================
img = mpimg.imread(map_img)
df_map = pd.read_csv(map_csv, sep=";", dtype=str)

for _, r in df_map.iterrows():
    est = r["estacao_obs"]
    cod = r["codigo_mini"]
    sub_bacia = r["Sub-bacia"]

    # >>> AJUSTE PEDIDO:
    # - nome_arquivo: usa _ (seguro p/ salvar arquivo)
    # - nome_titulo: sem _ (bonito p/ título da figura)
    nome_raw     = r["nome_estacao"]
    nome_arquivo = nome_raw.replace(" ", "_")
    nome_titulo  = nome_raw.replace("_", " ")

    obs = read_txt(inp_obs / f"{est}.txt")
    sim = read_txt(inp_sim / f"SIM_MC_{cod}.TXT")

    obs["q"] = obs["q"].replace(0, 1e-2)
    sim["q"] = sim["q"].replace(0, 1e-2)

    for rot, (ini, fim) in periodos.items():
        periodo_str = f"{ini[:4]}-{fim[:4]}"
        obs_p = obs.loc[ini:fim]
        sim_p = sim.loc[ini:fim]
        m = metrics(obs_p["q"], sim_p["q"])
        if m is None:
            print(f"⚠️ sem dados: {nome_arquivo} ({rot})")
            continue

        # ===== FIG =====
        fig = plt.figure(figsize=(cm2in(W_CM), cm2in(H_CM)))

        gs = fig.add_gridspec(1, 3, width_ratios=[3.3, 5.3, 4.4], wspace=0.2)

        ax0 = fig.add_subplot(gs[0,0])
        ax1 = fig.add_subplot(gs[0,1])
        ax2 = fig.add_subplot(gs[0,2])

        # mapa
        ax0.imshow(img, aspect="equal")
        ax0.set_axis_off()
        ax0.set_title(f"Sub-bacia {sub_bacia}", fontsize=FS_LAB, pad=2)

        # hidrograma
        ax1.plot(obs_p.index, obs_p["q"], color="black", lw=0.85)
        ax1.plot(sim_p.index, sim_p["q"], color="red", ls="--", lw=0.85)
        ax1.set_ylabel("Vazão (m³/s)", fontsize=FS_LAB, labelpad=4)
        ax1.grid(alpha=0.25)
        ax1.tick_params(labelsize=FS_TICK)

        stat_txt = "\n".join([f"{k}: {v:.2f}" for k,v in m.items()])
        ax1.text(0.01, 0.98, stat_txt, transform=ax1.transAxes, va="top", ha="left",
                 fontsize=FS_STAT, bbox=dict(facecolor="white", alpha=0.85, edgecolor="none", pad=2))

        # FDC
        p1,v1 = fdc(obs_p["q"]); p2,v2 = fdc(sim_p["q"])
        ax2.plot(p1, v1, color="black", lw=0.85)
        ax2.plot(p2, v2, color="red", lw=0.85)
        ax2.set_yscale("log")
        ax2.set_xlabel("Excedência (%)", fontsize=FS_LAB, labelpad=2)

        # ylabel do FDC à direita
        ax2.set_ylabel("Vazão (m³/s)", fontsize=FS_LAB, labelpad=4)
        ax2.yaxis.set_label_position("left")
        ax2.yaxis.tick_left()

        ax2.grid(alpha=0.25)
        ax2.tick_params(labelsize=FS_TICK)

        # >>> título único (agora sem _)
        fig.suptitle(f"{nome_titulo} – {est} – ({periodo_str})",fontsize=FS_TITLE,y=0.995)


        # legenda única embaixo
        h_obs = plt.Line2D([0],[0], color="black", lw=1.3)
        h_sim = plt.Line2D([0],[0], color="red", ls="--", lw=1.3)

        fig.subplots_adjust(left=0.02, right=0.98, top=0.92, bottom=0.26)

        fig.legend([h_obs, h_sim], ["Observado", "Simulado"], loc="lower center",
                   ncol=2, frameon=False, fontsize=FS_LAB, bbox_to_anchor=(0.5, 0.05),
                   handlelength=2.5, columnspacing=1.6)

        # >>> arquivo segue com _
        out_png = outdir / f"{nome_arquivo}_{rot}_painel_{periodo_str}.png"
        fig.savefig(out_png, dpi=DPI, bbox_inches="tight", pad_inches=0.02)
        plt.close(fig)

        print(f"✅ {out_png}")

In [ ]:
# -*- coding: utf-8 -*-
"""
===============================================================================
Estatísticas Descritivas de Vazão – Estação União da Vitória (65310000)
===============================================================================

Objetivo
--------
Calcular estatísticas descritivas das vazões observadas para dois períodos
históricos distintos (1931–2023 e 1980–2023), permitindo comparação direta
entre um período longo e um período recente.

Arquivos de entrada
-------------------
- 65310000_1931-2023.csv
- 65310000_1980_2023.csv

Formato esperado
----------------
CSV com separador ";" e sem cabeçalho, contendo:
dia;mes;ano;vazao_m3s

Saída
-----
Arquivo CSV único com tabela comparativa, pronto para inserção na dissertação.
"""

import pandas as pd
from pathlib import Path

# ======================================================
# CAMINHOS DE ENTRADA
# ======================================================

file_1931_2023 = Path(
    r"E:\IGUAÇU_OTTO\3_Estações FLU\Estatisticas\65310000_1931-2023.csv"
)

file_1980_2023 = Path(
    r"E:\IGUAÇU_OTTO\3_Estações FLU\Estatisticas\65310000_1980_2023.csv"
)

# ======================================================
# LEITURA DOS DADOS (SEM HEADER)
# ======================================================

cols = ["dia", "mes", "ano", "vazao_m3s"]

df_1931 = pd.read_csv(file_1931_2023, sep=";", header=None, names=cols)
df_1980 = pd.read_csv(file_1980_2023, sep=";", header=None, names=cols)

# Garante vazão numérica
df_1931["vazao_m3s"] = pd.to_numeric(df_1931["vazao_m3s"], errors="coerce")
df_1980["vazao_m3s"] = pd.to_numeric(df_1980["vazao_m3s"], errors="coerce")

# ======================================================
# FUNÇÃO DE ESTATÍSTICAS DESCRITIVAS
# ======================================================

def estatisticas_descritivas(serie):
    serie = serie.dropna()

    return {
        "Mínimo (m³/s)": serie.min(),
        "P5 (m³/s)": serie.quantile(0.05),
        "Q1 (m³/s)": serie.quantile(0.25),
        "Mediana (m³/s)": serie.median(),
        "Média (m³/s)": serie.mean(),
        "Q3 (m³/s)": serie.quantile(0.75),
        "P95 (m³/s)": serie.quantile(0.95),
        "Máximo (m³/s)": serie.max(),
        "Amplitude (m³/s)": serie.max() - serie.min(),
        "Desvio-padrão (m³/s)": serie.std(),
        "Coef. de variação (%)": (serie.std() / serie.mean()) * 100,
        "IQR (m³/s)": serie.quantile(0.75) - serie.quantile(0.25),
        "Assimetria (-)": serie.skew(),
        "Curtose (-)": serie.kurtosis()
    }

# ======================================================
# CÁLCULO DAS ESTATÍSTICAS
# ======================================================

stats_1931 = estatisticas_descritivas(df_1931["vazao_m3s"])
stats_1980 = estatisticas_descritivas(df_1980["vazao_m3s"])

# ======================================================
# TABELA COMPARATIVA FINAL
# ======================================================

tabela_final = pd.DataFrame({
    "Estatística": stats_1931.keys(),
    "1931–2023": stats_1931.values(),
    "1980–2023": stats_1980.values()
})

# ======================================================
# EXPORTAÇÃO
# ======================================================

output_path = Path(
    r"E:\IGUAÇU_OTTO\3_Estações FLU\Estatisticas\65310000_Estatisticas_Comparativas.csv"
)

tabela_final.to_csv(output_path, index=False, sep=";")

print("Arquivo gerado com sucesso:")
print(output_path)

In [ ]:
# -*- coding: utf-8 -*-
"""
===============================================================================
Disponibilidade de dados - Precipitação (formato MGB) | PDF A4 paisagem
VERSÃO MELHORADA - Visualização aprimorada
+ Remove estações sem dados (0% disponibilidade)
+ Cores por faixa de disponibilidade anual
+ Linhas separadoras entre estações
+ Exportação PNG automática
===============================================================================
"""

from __future__ import annotations

import re
import shutil
from pathlib import Path
from typing import Dict, List, Tuple

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.dates as mdates
from matplotlib.colors import LinearSegmentedColormap, BoundaryNorm
from matplotlib.patches import Rectangle


# =============================================================================
# CONFIGURAÇÕES
# =============================================================================

INPUT_DIR = Path(r"E:\IGUAÇU_OTTO\3_Estações FLU\IAT\ascii_mgb")
OUTPUT_DIR = INPUT_DIR / "_disponibilidade"

FILE_GLOB = "*.txt"
STATIONS_PER_PAGE = 45
DPI = 300

# --- Remoção/Separação de estações sem dados ---
EMPTY_DIR = INPUT_DIR / "_sem_dados"      # destino
ACTION_EMPTY = "move"                     # "move" ou "copy"
EMPTY_THRESHOLD_PCT = 0.0                 # remove apenas 0% (nenhum dado válido)

# --- Exportação PNG ---
SAVE_PNG = True  # ✓ Habilitado por padrão
PNG_PREFIX = "disp_precip"

# --- Cores por faixa de disponibilidade ---
# Esquema: vermelho (0-25%), laranja (25-50%), amarelo (50-75%), verde (75-100%)
COLOR_SCHEME = {
    'no_data': '#FFFFFF',      # Branco para sem dados
    'very_low': '#D73027',     # Vermelho escuro (0-25%)
    'low': '#FC8D59',          # Laranja (25-50%)
    'medium': '#FEE090',       # Amarelo claro (50-75%)
    'high': '#91CF60',         # Verde claro (75-90%)
    'very_high': '#1A9850',    # Verde escuro (90-100%)
}


# =============================================================================
# LEITOR ROBUSTO
# =============================================================================

_NUM_RE = re.compile(r"[-+]?\d+(?:[.,]\d+)?")

def _to_float_token(tok: str) -> float:
    tok = tok.strip()
    if "," in tok and "." in tok:
        if tok.rfind(".") > tok.rfind(","):
            tok = tok.replace(",", "")
        else:
            tok = tok.replace(".", "").replace(",", ".")
        return float(tok)
    if "," in tok:
        return float(tok.replace(".", "").replace(",", "."))
    return float(tok)

def read_mgb_precip_file(fp: Path) -> pd.DataFrame:
    rows = []
    with fp.open("r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            nums = _NUM_RE.findall(line)
            if len(nums) < 4:
                continue
            try:
                day = int(float(nums[0].replace(",", ".")))
                month = int(float(nums[1].replace(",", ".")))
                year = int(float(nums[2].replace(",", ".")))
                value = _to_float_token(nums[-1])
                rows.append((day, month, year, value))
            except Exception:
                continue

    df = pd.DataFrame(rows, columns=["day", "month", "year", "value"])
    df["date"] = pd.to_datetime(
        dict(year=df["year"], month=df["month"], day=df["day"]),
        errors="coerce",
    )
    df = df.dropna(subset=["date"]).copy().sort_values("date")
    df["is_valid"] = df["value"].ge(0.0)  # 0 é válido; negativo é faltante
    return df[["date", "value", "is_valid"]]


# =============================================================================
# UTILITÁRIOS
# =============================================================================

def station_id_from_filename(fp: Path) -> str:
    return fp.stem

def chunk_list(items: List[str], chunk_size: int) -> List[List[str]]:
    return [items[i:i + chunk_size] for i in range(0, len(items), chunk_size)]

def format_period(dmin: pd.Timestamp, dmax: pd.Timestamp) -> str:
    return f"{dmin.strftime('%Y-%m-%d')}–{dmax.strftime('%Y-%m-%d')}"

def build_availability_matrix(
    series_by_station: Dict[str, pd.Series],
    global_dates: pd.DatetimeIndex
) -> Tuple[np.ndarray, List[str], np.ndarray, np.ndarray]:
    """
    Retorna:
        mat: matriz de disponibilidade diária (0/1)
        station_ids: lista de IDs das estações
        pct_valid: % de disponibilidade total
        annual_pct: matriz de % de disponibilidade anual (estações x anos)
    """
    station_ids = sorted(series_by_station.keys())
    mat = np.zeros((len(station_ids), len(global_dates)), dtype=np.uint8)
    pct_valid = np.zeros(len(station_ids), dtype=float)
    
    # Calcular disponibilidade anual
    years = global_dates.year.unique()
    annual_pct = np.full((len(station_ids), len(global_dates)), np.nan, dtype=float)

    for i, sid in enumerate(station_ids):
        s = series_by_station[sid].reindex(global_dates, fill_value=False)
        a = s.astype(np.uint8).to_numpy()
        mat[i, :] = a
        pct_valid[i] = 100.0 * a.mean() if len(a) else 0.0
        
        # Calcular % por ano para cada data
        for j, date in enumerate(global_dates):
            year = date.year
            year_mask = global_dates.year == year
            year_data = a[year_mask]
            if len(year_data) > 0:
                annual_pct[i, j] = 100.0 * year_data.mean()

    return mat, station_ids, pct_valid, annual_pct

def get_color_for_availability(pct: float) -> str:
    """Retorna cor baseada na % de disponibilidade"""
    if np.isnan(pct) or pct == 0:
        return COLOR_SCHEME['no_data']
    elif pct < 25:
        return COLOR_SCHEME['very_low']
    elif pct < 50:
        return COLOR_SCHEME['low']
    elif pct < 75:
        return COLOR_SCHEME['medium']
    elif pct < 90:
        return COLOR_SCHEME['high']
    else:
        return COLOR_SCHEME['very_high']

def plot_page(
    mat: np.ndarray,
    station_ids: List[str],
    pct_valid: np.ndarray,
    annual_pct: np.ndarray,
    global_dates: pd.DatetimeIndex,
    title: str,
    subtitle: str,
) -> plt.Figure:
    """Plota uma página com visualização melhorada"""
    
    # Tamanho A4 paisagem em polegadas (297mm x 210mm)
    fig = plt.figure(figsize=(11.69, 8.27), dpi=DPI)

    # Layout com 3 colunas: principal, barra %, legenda
    gs = fig.add_gridspec(
        nrows=1, ncols=3,
        width_ratios=[15, 2.5, 2],
        left=0.05, right=0.98, top=0.88, bottom=0.12,
        wspace=0.15
    )
    
    ax_main = fig.add_subplot(gs[0, 0])
    ax_bar = fig.add_subplot(gs[0, 1], sharey=ax_main)
    ax_legend = fig.add_subplot(gs[0, 2])
    ax_legend.axis('off')

    # === PLOT PRINCIPAL com cores por disponibilidade anual ===
    x0 = mdates.date2num(global_dates[0].to_pydatetime())
    x1 = mdates.date2num(global_dates[-1].to_pydatetime())
    
    # Criar imagem RGB colorida baseada na disponibilidade anual
    img_rgb = np.ones((len(station_ids), len(global_dates), 3))
    
    for i in range(len(station_ids)):
        for j in range(len(global_dates)):
            if mat[i, j] == 0:
                # Sem dado: branco
                color_hex = COLOR_SCHEME['no_data']
            else:
                # Com dado: cor baseada na disponibilidade anual
                pct = annual_pct[i, j]
                color_hex = get_color_for_availability(pct)
            
            # Converter hex para RGB normalizado
            color_hex = color_hex.lstrip('#')
            rgb = tuple(int(color_hex[k:k+2], 16) / 255.0 for k in (0, 2, 4))
            img_rgb[i, j, :] = rgb

    ax_main.imshow(
        img_rgb,
        aspect="auto",
        interpolation="none",
        extent=(x0, x1, 0, len(station_ids)),
        origin="lower",
    )

    # === LINHAS HORIZONTAIS separando estações ===
    for i in range(len(station_ids) + 1):
        ax_main.axhline(y=i, color='0.7', linewidth=0.3, alpha=0.5, zorder=10)

    # === TÍTULOS E LABELS ===
    fig.suptitle(title, fontsize=13, fontweight='bold', y=0.955)
    fig.text(0.05, 0.905, subtitle, fontsize=9, style='italic')

    # Eixo X - Anos
    ax_main.xaxis_date()
    ax_main.xaxis.set_major_locator(mdates.YearLocator(base=5))
    ax_main.xaxis.set_major_formatter(mdates.DateFormatter("%Y"))
    ax_main.xaxis.set_minor_locator(mdates.YearLocator(base=1))
    ax_main.set_xlabel("Ano", fontsize=10, fontweight='bold')
    
    # Eixo Y - Estações
    ax_main.set_ylabel("Código da Estação", fontsize=10, fontweight='bold')
    ax_main.set_yticks(np.arange(len(station_ids)) + 0.5)
    ax_main.set_yticklabels(station_ids, fontsize=6, family='monospace')

    # Grid vertical leve
    ax_main.grid(which="major", axis="x", linewidth=0.4, alpha=0.3, color='0.4')
    ax_main.grid(which="minor", axis="x", linewidth=0.2, alpha=0.2, color='0.6')

    # === BARRA DE DISPONIBILIDADE TOTAL ===
    y = np.arange(len(station_ids)) + 0.5
    colors_bar = [get_color_for_availability(p) for p in pct_valid]
    
    bars = ax_bar.barh(y=y, width=pct_valid, height=0.85, 
                       color=colors_bar, edgecolor='0.3', linewidth=0.5)
    
    ax_bar.set_xlim(0, 100)
    ax_bar.set_xlabel("Disponibilidade\nTotal (%)", fontsize=9, fontweight='bold')
    ax_bar.set_xticks([0, 25, 50, 75, 100])
    ax_bar.tick_params(axis="x", labelsize=8)
    ax_bar.tick_params(axis="y", left=False, labelleft=False)
    ax_bar.grid(axis='x', alpha=0.3, linewidth=0.3)
    
    # Adicionar valores de % nas barras (para valores > 15%)
    for i, (bar, pct) in enumerate(zip(bars, pct_valid)):
        if pct > 15:
            ax_bar.text(pct - 2, y[i], f'{pct:.0f}', 
                       ha='right', va='center', fontsize=5, 
                       color='white', fontweight='bold')

    # === LEGENDA DE CORES ===
    ax_legend.set_xlim(0, 1)
    ax_legend.set_ylim(0, 1)
    
    legend_items = [
        (COLOR_SCHEME['very_high'], '90-100%', 0.85),
        (COLOR_SCHEME['high'], '75-90%', 0.70),
        (COLOR_SCHEME['medium'], '50-75%', 0.55),
        (COLOR_SCHEME['low'], '25-50%', 0.40),
        (COLOR_SCHEME['very_low'], '0-25%', 0.25),
        (COLOR_SCHEME['no_data'], 'Sem dado', 0.08),
    ]
    
    ax_legend.text(0.5, 0.98, 'Disponibilidade\nAnual', 
                  ha='center', va='top', fontsize=9, fontweight='bold')
    
    for color, label, y_pos in legend_items:
        rect = Rectangle((0.1, y_pos - 0.04), 0.25, 0.06, 
                        facecolor=color, edgecolor='0.3', linewidth=0.8)
        ax_legend.add_patch(rect)
        ax_legend.text(0.4, y_pos, label, va='center', fontsize=8)

    # === BORDAS ===
    for a in [ax_main, ax_bar]:
        a.spines['top'].set_linewidth(1.2)
        a.spines['right'].set_linewidth(1.2)
        a.spines['bottom'].set_linewidth(1.2)
        a.spines['left'].set_linewidth(1.2)

    # === NOTA DE RODAPÉ ===
    fig.text(
        0.05, 0.06,
        "Nota: As cores representam a disponibilidade de dados válidos (≥ 0 mm) em cada ano. "
        "Valores negativos são considerados ausência de dado.",
        fontsize=7, style='italic', color='0.3'
    )
    
    fig.text(
        0.05, 0.03,
        f"Gerado em: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M')}",
        fontsize=6, color='0.5'
    )
    
    return fig

def relocate_empty_file(src: Path, dest_dir: Path, action: str = "move") -> Path:
    dest_dir.mkdir(parents=True, exist_ok=True)
    dst = dest_dir / src.name
    if action == "move":
        shutil.move(str(src), str(dst))
    elif action == "copy":
        shutil.copy2(str(src), str(dst))
    else:
        raise ValueError("ACTION_EMPTY deve ser 'move' ou 'copy'")
    return dst


# =============================================================================
# MAIN
# =============================================================================

def main() -> None:
    OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

    files = sorted(INPUT_DIR.glob(FILE_GLOB))
    if not files:
        raise FileNotFoundError(f"Nenhum arquivo encontrado em {INPUT_DIR} com padrão {FILE_GLOB}")

    # Separar estações sem dado
    series_by_station: Dict[str, pd.Series] = {}
    kept_files = []
    moved = []
    summary_rows = []

    print("Processando arquivos...")
    for fp in files:
        sid = station_id_from_filename(fp)
        df = read_mgb_precip_file(fp)

        # Caso 1: arquivo sem linhas úteis
        if df.empty:
            reason = "arquivo sem linhas úteis"
            moved.append((fp.name, reason))
            relocate_empty_file(fp, EMPTY_DIR, ACTION_EMPTY)

            summary_rows.append({
                "station_id": sid,
                "arquivo": fp.name,
                "status": "sem_dados",
                "motivo": reason,
                "data_inicio": "",
                "data_fim": "",
                "n_dias_total": 0,
                "n_dias_validos": 0,
                "pct_dias_validos": 0.0,
            })
            continue

        # Agrega por dia: válido se existir ao menos um registro >=0 no dia
        daily_valid = df.groupby("date")["is_valid"].any()

        # métricas no período do próprio arquivo
        dmin = daily_valid.index.min()
        dmax = daily_valid.index.max()
        n_total = int(daily_valid.shape[0])
        n_valid = int(daily_valid.sum())
        pct_here = (100.0 * (n_valid / n_total)) if n_total > 0 else 0.0

        # Caso 2: 0 dias válidos
        if daily_valid.any() is False:
            reason = "0 dias válidos (>=0)"
            moved.append((fp.name, reason))
            relocate_empty_file(fp, EMPTY_DIR, ACTION_EMPTY)

            summary_rows.append({
                "station_id": sid,
                "arquivo": fp.name,
                "status": "sem_dados",
                "motivo": reason,
                "data_inicio": dmin.date().isoformat() if pd.notna(dmin) else "",
                "data_fim": dmax.date().isoformat() if pd.notna(dmax) else "",
                "n_dias_total": n_total,
                "n_dias_validos": n_valid,
                "pct_dias_validos": round(pct_here, 4),
            })
            continue

        # Caso 3: abaixo do limiar
        if pct_here <= EMPTY_THRESHOLD_PCT:
            reason = f"pct={pct_here:.2f}% <= limiar"
            moved.append((fp.name, reason))
            relocate_empty_file(fp, EMPTY_DIR, ACTION_EMPTY)

            summary_rows.append({
                "station_id": sid,
                "arquivo": fp.name,
                "status": "sem_dados",
                "motivo": reason,
                "data_inicio": dmin.date().isoformat() if pd.notna(dmin) else "",
                "data_fim": dmax.date().isoformat() if pd.notna(dmax) else "",
                "n_dias_total": n_total,
                "n_dias_validos": n_valid,
                "pct_dias_validos": round(pct_here, 4),
            })
            continue

        # Caso 4: estação plotada (mantida)
        series_by_station[sid] = daily_valid
        kept_files.append(fp)

        summary_rows.append({
            "station_id": sid,
            "arquivo": fp.name,
            "status": "plotada",
            "motivo": "",
            "data_inicio": dmin.date().isoformat() if pd.notna(dmin) else "",
            "data_fim": dmax.date().isoformat() if pd.notna(dmax) else "",
            "n_dias_total": n_total,
            "n_dias_validos": n_valid,
            "pct_dias_validos": round(pct_here, 4),
        })

    # Salvar CSV resumo
    print("Gerando CSV resumo...")
    summary_csv_path = OUTPUT_DIR / "resumo_disponibilidade_estacoes.csv"
    df_summary = pd.DataFrame(summary_rows)

    if not df_summary.empty:
        df_summary["status_ord"] = df_summary["status"].map({"plotada": 0, "sem_dados": 1}).fillna(9)
        df_summary = df_summary.sort_values(
            by=["status_ord", "pct_dias_validos", "station_id"],
            ascending=[True, False, True]
        ).drop(columns=["status_ord"])

    df_summary.to_csv(summary_csv_path, index=False, encoding="utf-8-sig", sep=";")
    print(f"✓ CSV resumo: {summary_csv_path}")

    if not series_by_station:
        raise RuntimeError("Após filtrar estações sem dados, não restou nenhuma estação para plotar.")

    # Período global
    min_date = min(s.index.min() for s in series_by_station.values())
    max_date = max(s.index.max() for s in series_by_station.values())
    global_dates = pd.date_range(min_date, max_date, freq="D")

    all_station_ids = sorted(series_by_station.keys())
    pages = chunk_list(all_station_ids, STATIONS_PER_PAGE)

    # Gerar PDF e PNG
    print(f"Gerando visualizações ({len(pages)} páginas)...")
    pdf_path = OUTPUT_DIR / "disponibilidade_precipitacao_MGB_A4_paisagem.pdf"
    
    with PdfPages(pdf_path) as pdf:
        for p, station_chunk in enumerate(pages, start=1):
            print(f"  Página {p}/{len(pages)}...", end=" ")
            
            subdict = {sid: series_by_station[sid] for sid in station_chunk}
            mat, station_ids, pct_valid, annual_pct = build_availability_matrix(subdict, global_dates)

            title = "Disponibilidade de Dados de Precipitação (Formato MGB)"
            subtitle = (
                f"Página {p}/{len(pages)} | Estações {((p-1)*STATIONS_PER_PAGE)+1}"
                f"–{((p-1)*STATIONS_PER_PAGE)+len(station_ids)} de {len(all_station_ids)} | "
                f"Período: {format_period(min_date, max_date)}"
            )

            fig = plot_page(mat, station_ids, pct_valid, annual_pct, 
                          global_dates, title, subtitle)
            
            # Salvar no PDF
            pdf.savefig(fig, dpi=DPI, bbox_inches='tight')
            
            # Salvar PNG
            if SAVE_PNG:
                png_path = OUTPUT_DIR / f"{PNG_PREFIX}_p{p:03d}.png"
                fig.savefig(png_path, dpi=DPI, bbox_inches='tight')
                print(f"✓ PNG salvo")
            else:
                print("✓")
            
            plt.close(fig)

    print(f"\n{'='*70}")
    print(f"✓ PDF gerado: {pdf_path}")
    print(f"✓ Estações plotadas: {len(all_station_ids)}")
    print(f"✓ Estações sem dados relocadas: {len(moved)} → {EMPTY_DIR}")
    
    if SAVE_PNG:
        print(f"✓ Arquivos PNG salvos em: {OUTPUT_DIR}")
    
    print(f"{'='*70}")

    if moved and len(moved) <= 10:
        print("\nArquivos relocados (sem dados):")
        for name, reason in moved:
            print(f"  • {name} | {reason}")
    elif moved:
        print(f"\n{len(moved)} arquivos relocados. Veja detalhes no CSV resumo.")


if __name__ == "__main__":
    main()